### Multitenancy in Azure Land

1. Create a multitenant app using Azure's app registration (Preview)

    * There is not a good way to do this with the CLI/API, we've got to register the app with the UI. It could just be because this is still in app preview. Note: ask Kyle if adding CLI/API support is on the MSFT roadmap.
    
2. Have the user consent through an `adminconsent` endpoint. The required parameters to this are:
    
    * cloudigrade's client_id (found in azure app registration)
    * the redirect uri (should match what's set in azure app registration)
    * the scope. To get the resources we need access to the management API, there is /no/ fine grained permissions for that, only user_impersonation.

    `https://login.microsoftonline.com/common/adminconsent?client_id=836ace98-9581-4ff3-a866-76f266186252&response_mode=query&response_type=code&redirect_uri=http%3a%2f%2f127.0.0.1:8000/oidc/callback/&scope=https://management.azure.com/user_impersonation`

    The response from this call includes the client's tenant id, which we will need later.

    `http://127.0.0.1:8000/oidc/callback/?admin_consent=True&tenant=52036cf3-b413-415c-b819-14c27e049e16`
    
    Note: An error occurs when logging in with some accounts. The following occurs when logging in with my `bihanzh2@outlook.com` account, but does not occurs when logging in with a created `testuser@bihanzh2outlook.onmicrosoft.com`. I /think/ the difference is latter is an Azure Directory Account, and the former is an external account that I'm using to access azure.
    
    `http://127.0.0.1:8000/oidc/callback/?error=access_denied&error_description=AADSTS50020%3a+User+account+%27XXX%27+from+identity+provider+%27live.com%27+does+not+exist+in+tenant+%27Default+Directory%27+and+cannot+access+the+application+%27836ace98-9581-4ff3-a866-76f266186252%27+in+that+tenant.++The+account+needs+to+be+added+as+an+external+user+in+the+tenant+first.++Sign+out+and+sign+in+again+with+a+different+Azure+Active+Directory+user+account.%0d%0aTrace+ID%3a+b6b847dc-7646-494c-be04-d041110d2b00%0d%0aCorrelation+ID%3a+cb2e3b3f-707f-406a-bb71-733608fe1e2d%0d%0aTimestamp%3a+2018-12-21+21%3a28%3a08Z&admin_consent=True&tenant=85fa570e-8b49-4eed-8bdf-3f3a72c89932`
    
3. When the user grants cloudigrade access through the `adminconsent` call a service provider is created on that user account. This sp needs to have roles granted to it before we can actually use it to access their compute resources. We can do this using the Graph API


4. (Finally) we are able to get tokens whenever we want to make a request. And can use the token to get the user's resources. 

In [ ]:
#Access via API calls:
import requests
import json

from getpass import getpass
from pprint import pprint

redirect_uri = 'http://127.0.0.1:8000/oidc/callback/'

# this is brad's tenant
brad_tenant = '85fa570e-8b49-4eed-8bdf-3f3a72c89932'

# this is the user tenant
bizhang_tenant = '52036cf3-b413-415c-b819-14c27e049e16'
bizhang_subscription = 'c448f2a4-1542-421a-b860-d6ea2182e746'

# this is brad's multitenant application client id
client_id = '836ace98-9581-4ff3-a866-76f266186252'
client_secret = getpass()

payload = {'grant_type': 'client_credentials',           
           'redirect_uri': redirect_uri, 
           'client_secret': client_secret,
           'client_id': client_id,
           'scope': [
               #'https://management.azure.com/.default',
               'https://graph.microsoft.com/.default'
           ]}

r = requests.post(
    'https://login.microsoftonline.com/{}/oauth2/v2.0/token'.format(bizhang_tenant),
    data=payload
)

pprint(r.json())
access_token=r.json()['access_token']


In [184]:
# We need to user the permissions to the graph API to assign a role to our service principal on the client subscription

# Get service principal id:
resp = requests.get( 
    'https://graph.microsoft.com/beta/servicePrincipals/',
    headers={
        "Authorization": "Bearer {}".format(access_token)
    }
)
sp_id = list(filter(lambda app: app['appId'] == client_id, resp.json()['value']))[0]['id']

payload = {
    'appRoleId': '5778995a-e1bf-45b8-affa-663a9f3f4d04',
    'id': 'LFAeMnGBn0uCploZ9cDDZarzkGCpaHpPqQqcqU5ZvI4',
    'principalDisplayName': 'brads_multitenant_app',
    'principalId': '321e502c-8171-4b9f-82a6-5a19f5c0c365',
    'principalType': 'ServicePrincipal',
    'resourceDisplayName': 'Windows Azure Active Directory',
    'resourceId': '68ba5fb1-4080-42cd-8a27-050f643db9dd'
}
resp = requests.get(
    #'https://management.azure.com/subscriptions/c448f2a4-1542-421a-b860-d6ea2182e746/resources?api-version=2018-11-01',
    #'https://management.azure.com/subscriptions?api-version=2018-11-01',
    'https://graph.microsoft.com/beta/servicePrincipals/{id}/appRoleAssignedTo/'.format(id=sp_id),
    headers={
        "Authorization": "Bearer {}".format(access_token)
    },
    data=payload,
)

pprint(resp.json())

{'@odata.context': 'https://graph.microsoft.com/beta/$metadata#appRoleAssignments',
 'value': [{'appRoleId': '5778995a-e1bf-45b8-affa-663a9f3f4d04',
            'creationTimestamp': '2018-12-14T16:31:14.1581783Z',
            'id': 'LFAeMnGBn0uCploZ9cDDZarzkGCpaHpPqQqcqU5ZvI4',
            'principalDisplayName': 'brads_multitenant_app',
            'principalId': '321e502c-8171-4b9f-82a6-5a19f5c0c365',
            'principalType': 'ServicePrincipal',
            'resourceDisplayName': 'Windows Azure Active Directory',
            'resourceId': '68ba5fb1-4080-42cd-8a27-050f643db9dd'},
           {'appRoleId': '741f803b-c850-494e-b5df-cde7c675a1ca',
            'creationTimestamp': '2018-12-14T16:31:13.9381478Z',
            'id': 'LFAeMnGBn0uCploZ9cDDZa7AZn88WkBOsZD2v-DA7Lg',
            'principalDisplayName': 'brads_multitenant_app',
            'principalId': '321e502c-8171-4b9f-82a6-5a19f5c0c365',
            'principalType': 'ServicePrincipal',
            'resourceDisplayName': '

In [107]:
#Once set up is done we can get access tokens and get resource information with it.

import adal

from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from msrestazure.azure_active_directory import AdalAuthentication
from msrestazure.azure_cloud import AZURE_PUBLIC_CLOUD

LOGIN_ENDPOINT = AZURE_PUBLIC_CLOUD.endpoints.active_directory
RESOURCE = AZURE_PUBLIC_CLOUD.endpoints.active_directory_resource_id

context = adal.AuthenticationContext(LOGIN_ENDPOINT + '/' + bizhang_tenant)
credentials = AdalAuthentication(
    context.acquire_token_with_client_credentials,
    RESOURCE,
    client_id,
    client_secret
)

resource_client = ResourceManagementClient(credentials, bizhang_subscription)
compute_client = ComputeManagementClient(credentials, bizhang_subscription)

In [108]:
# resource_client.resources does not include resource group info 
# which we need to get to the vm, so we have to start by getting all the groups
for groups in resource_client.resource_groups.list(): 
    resources = resource_client.resources.list_by_resource_group(groups.name)
    for resource in resources:
        pprint(resource.name)

'blehstorage'
